In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import os
import requests, re, string
import difflib
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory as SF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
# Read dataset to get the result
df_resep = pd.read_csv('resep_final.csv')

# Read dataset for processing
df_cleaned = pd.read_csv('df_cleaned.csv')
df_cleaned['tags'] = df_cleaned['menu'] + ' ' + df_cleaned['bahan'] + ' ' + df_cleaned['langkah']

In [45]:
# Make object Tfidf and Cosine Similarity for searching the desired resep
vectorizer = TfidfVectorizer(min_df=1, ngram_range=(1,2))
features_vectors = vectorizer.fit_transform(df_cleaned['tags'])
similarity = cosine_similarity(features_vectors)

In [47]:
# Function to clean the user input
def case_folding(data):
    data = data.lower()
    data = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",data).split())
    data = re.sub(r"\d+", "", data)
    data = data.translate(str.maketrans("","",string.punctuation))
    data = re.sub(r"\n","",data)
    data = re.sub(r"\t","",data)
    return data

def stopword_cleaner(data):
    sw_indonesia = stopwords.words("indonesian")
    sw_indonesia.remove('tahu')
    data  = [word for word in data if word not in sw_indonesia]
    data = ' '.join(data)
    return data

In [64]:
# Input user (ini perintahnya bisa diganti sabeb, trs inputannya ga case sensitive karena udh dihandle,
# intinya inputan harus bahan, co: Telur, kacang AYAM | ladA, 'PISANG')
kata_kunci = input('Masukkan bahan yang kamu punya: ')

Masukkan bahan yang kamu punya: 


In [65]:
# Function to process user input and get the result
def rekomendasi(keywords):
    # Search the input that quite similar to dataset 
    similarity_scores = [difflib.SequenceMatcher(None, keywords, tags).ratio() for tags in df_cleaned['bahan'].tolist()]
    closest_match_index = similarity_scores.index(max(similarity_scores))
    similarity_score = sorted(list(enumerate(similarity[closest_match_index])), key=lambda x:x[1], reverse=True)
    
    # Return the data (ini formatnya dataframe)
    df_result = pd.DataFrame(columns=['menu', 'bahan', 'langkah', 'hasil', 'kalori_kkal', 'lemak_g','karbohidrat_g', 'protein_g', 'serat_g', 'kolesterol_mg', 'gambar_url'])
    i = 1
    for item in similarity_score:
        index = item[0]
        if(i<10):
            df_result.loc[index] = df_resep.loc[index]
            i+=1
    return df_result

In [61]:
# # Ini return nama-nama menunya doang (opsional/kalo butuh)
# def rekomendasi(keywords):
#     # Search the input that quite similar to dataset 
#     similarity_scores = [difflib.SequenceMatcher(None, keywords, tags).ratio() for tags in df_cleaned['bahan'].tolist()]
#     closest_match_index = similarity_scores.index(max(similarity_scores))
#     similarity_score = sorted(list(enumerate(similarity[closest_match_index])), key=lambda x:x[1], reverse=True)
    
#     # Return the data (ini format dataframe)
#     list_menu = []
#     i = 1
#     for item in similarity_score:
#         index = item[0]
#         menu_from_index = df_cleaned[df_cleaned.index==index]['menu'].values[0]
#         if(i<11):
#             list_menu.append(menu_from_index)
#             i+=1
#     return list_menu

# rekomendasi(kata_kunci)